## Part 2: MCMC原理

使用MCMC的原因：
- 由于贝叶斯公式分母难以计算
- 由于参数过多后验分布形态复杂

在本节课中，我们首先回顾如何计算后验，以及计算后验面临的问题；

之后，我们介绍如何通过采样的方法解决现有的问题。

### 计算后验面临的问题

**以探究地球海水覆盖率为例。**

- 假设我们获得数据：抛地球仪10次，其中7次海面朝上。
- 在获得数据的同时，我们以二项分布函数作为**似然函数**。
- 假设我们在获得数据以前的**先验**认为：地球海水覆盖率为0.5。

我们的任务是根据数据更新我们的先验。

1. 假设我们获得数据：抛地球仪10次，其中7次海面朝上。

In [1]:
import numpy as np               # numpy 是专门用于数组运算的包
import scipy.stats as st         # 从scipy.stats里载入分布函数
import matplotlib.pyplot as plt  # matplotlib.pyplot 是专门用于画图的包
import seaborn as sns            # seaborn是专门用于绘制统计数据的包
import arviz as az               # arviz 是专门用于探索贝叶斯模型的包

In [2]:
data = [1,1,1,1,1,0,0,1,1,0] # 1代表海面朝上，0代表陆地朝上
print("数据：",data)

数据： [1, 1, 1, 1, 1, 0, 0, 1, 1, 0]


In [3]:
plt.hist(data)
plt.xticks([0,1],labels=["earth","water"])
plt.xlim(-1,2)

(-1.0, 2.0)

<Figure size 432x288 with 1 Axes>

2. 我们的先验：地球海水覆盖率为0.5。
  
由于我们的先验不是分布，而是一个值，因此可以认为我们对先验的信心为100%，即 $prior=1$

In [4]:
prior_θ = 0.5
prior=1 # 由于先验只有一个值，我们用1(100%)表示先验出现的概率
print("先验：地球海水覆盖率为：",prior_θ)

先验：地球海水覆盖率为： 0.5


3. 结合先验参数(prior_θ)与数据(data)计算似然值。

我们选择伯努利分布函数(bernoulli)作为似然函数。

In [5]:
likelihood  = st.bernoulli(prior_θ).pmf(data).prod()
print("根据当前数据与先验得到的似然值：",likelihood)

根据当前数据与先验得到的似然值： 0.0009765625


4. 计算后验，即通过似然值更新先验。

注意：

由于这里是没有考虑贝叶斯公式的分母部分，因此我们得到的是未标准化的 invalid 后验

In [6]:
unstd_posterior = likelihood * prior
print("未标准化的后验：",unstd_posterior)

未标准化的后验： 0.0009765625


**练习：**

当我们假设的先验不同时，似然与后验也会产生相应的变化。

比如，我们尝试将先验(地球海水覆盖率)设为0.5，0.6，和0.7。

- 为了后面方便计算，我们把计算未标准化后验的过程用一个函数包裹起来

该函数的每一步计算过程与之前完全相同

In [7]:
def calculate_posterior(prior_θ, data):
    prior=1
    likelihood  = st.bernoulli(prior_θ).pmf(data).prod()
    unstd_posterior = likelihood * prior
    
    return unstd_posterior

In [8]:
prior_θ = 0.5 # 假设先验为0.5
unstd_posterior = calculate_posterior(prior_θ,data)
print("未标准化的后验：",unstd_posterior)

未标准化的后验： 0.0009765625


In [ ]:
import numpy as np               # numpy 是专门用于数组运算的包
import scipy.stats as st         # 从scipy.stats里载入分布函数

# 定义计算后验的函数
def calculate_posterior(prior_θ, data):
    prior=1
    likelihood  = st.bernoulli(prior_θ).pmf(data).prod()
    unstd_posterior = likelihood * prior
    return unstd_posterior
  
########################################################
# 练习
# 请尝试修改先验参数 prior_θ 修改为0.6或者0.7
########################################################

prior_θ = ...
unstd_posterior = calculate_posterior(prior_θ,data)
print("未标准化的后验：",unstd_posterior)

通过定义函数`calculate_posterior`，我们只需要输入先验参数和数据，就可以计算出后验后验(invalid)。

经过尝试可以发现，当我们的先验参数越接近数据中海面出现的频率(0.7)时，后验值最大。

但此时的后验值还很小，还不是一个有效的概率值，接下来我们结合 **有效的先验概率分布**和**贝叶斯公式分母(边际似然)**计算有效的后验概率分布。

**计算valid 后验概率分布**

如果我们假设，海水覆盖面率可能不止是一个值，而可能为0.6，0.7，0.8，其中0.7的可能性最大时：

比如，我们认为海水覆盖率为0.7时的概率为0.5，而海水覆盖率为0.6和0.7的概率为0.25。

此时先验概率的和为1，所以该分布是一个有效的(valid)概率分布。

In [10]:
prior_θs = [0.6,0.7,0.8] # 假设先验为0.6，0.7，0.8
priors = [0.25,0.5,0.25] # 假设我们对不同先验的信心为0.25,0.5,0.25

In [11]:
# 绘制先验分布的概率分布
plt.plot(prior_θs,priors)
plt.xlabel("Prior belief of the proportion of sea surface coverage")
plt.ylabel("Probability")
plt.xticks(prior_θs)

([<matplotlib.axis.XTick at 0x7f0a38d59390>,
 [Text(0, 0, ''), Text(0, 0, ''), Text(0, 0, '')])

<Figure size 432x288 with 1 Axes>

接下来我们计算后验概率：
- `calculate_posterior` 是我们定义的计算未标准化的后验
- 因为先验有多个值(0.6,0.7,0.8)，我们通过 for 循环分别计算每一个先验值的后验：
  
  `for prior_θ,prior in zip(prior_θs,priors)`

  因为每一个先验参数有不同的概率，我们通过`zip`函数将两个变量打包在一起，使得可以同时循环两个变量。
- 为了保存循环计算得到的多个后验值，我们把各值加入到一个空数组`unstd_posteriors = []`

  即下面代码`unstd_posteriors += [unstd_posterior]`

In [12]:
# 定义计算后验的函数
def calculate_posterior(prior_θ, prior, data):
    likelihood  = st.bernoulli(prior_θ).pmf(data).prod()
    unstd_posterior = likelihood * prior
    return unstd_posterior

unstd_posteriors = []
for prior_θ,prior in zip(prior_θs,priors):
    unstd_posterior = calculate_posterior(prior_θ,prior,data)
    unstd_posteriors += [unstd_posterior]

Tips: 上面的循环可以写作下面**更简洁**的形式，这得力于python的语法糖。

In [13]:
unstd_posteriors = [calculate_posterior(prior_θ,prior,data) for prior_θ,prior in zip(prior_θs,priors)] 

In [14]:
print("未标准化的后验：",unstd_posteriors)

未标准化的后验： [0.00044789760000000004, 0.0011117830499999999, 0.0004194303999999999]


此时的后验不是有效的概率分布，我们可以通过计算贝叶斯公式中的分母，从而得到出标准化(valid)的后验概率。

In [15]:
posteriors = unstd_posteriors/np.sum(unstd_posteriors)
print("标准化的后验：",posteriors)

标准化的后验： [0.22631252 0.5617588  0.21192868]


可以看到，标准化的后验概率总和为1。

并且参数为0.7时，后验概率最大。数据增强了我们对于参数等于0.7的信心(之前我们认为参数等于0.7的概率为0.5)。

In [16]:
plt.plot(prior_θs,posteriors)
plt.xlabel("θ")
plt.ylabel("posterior probability")

Text(0, 0.5, 'posterior probability')

<Figure size 432x288 with 1 Axes>

**先验为正态分布时，计算后验分布**

同样，我们可以根据上节课的知识，为先验设置一个正态分布，然后再计算后验分布。

我们假设先验参数服从均值为0.5，标准差为0.1的正态分布

In [17]:
prior_θs = np.linspace(0, 1, 100) # 遍历先验参数值从0到1
priors = st.norm.pdf(prior_θs,0.5,0.1)/np.sum(st.norm.pdf(prior_θs,0.5,0.1)) # 先验参数服从均值为0.5，标准差为0.1的正态分布

In [18]:
plt.plot(prior_θs,priors)
plt.xlabel("prior θ")
plt.ylabel("priors probability")

Text(0, 0.5, 'priors probability')

<Figure size 432x288 with 1 Axes>

我们通过上面定义的计算后验的函数`calculate_posterior` 以及刚提到的 for循环计算每一个先验对应的后验值

In [19]:
unstd_posterior = [calculate_posterior(prior_θ,prior,data) for prior_θ,prior in zip(prior_θs,priors)]

In [20]:
plt.plot(prior_θs,unstd_posterior)
plt.xlabel("θ")
plt.ylabel("unstandardized posteriors")

Text(0, 0.5, 'unstandardized posteriors')

<Figure size 432x288 with 1 Axes>

注意，此时y轴表示的不是后验概率。

因此我们需要根据公式 $P(data) =\sum_{\theta}^{} p(data,\theta)=\sum_{\theta}^{} p(data|\theta)p(\theta)$ 计算后验概率。

对应的代码 

- $P(data) =$ `np.sum(unstd_posterior)`
- $p(\theta|data) = \frac{p(data|\theta)p(\theta)}{p(data)}$ 对应 `posterior = unstd_posterior/np.sum(unstd_posterior)`

In [21]:
posterior = unstd_posterior/np.sum(unstd_posterior)

plt.plot(prior_θs, priors, color = 'grey',ls = '--',label="prior")
plt.plot(prior_θs, posterior, color = 'red',label="posterior")
plt.legend()

<Figure size 432x288 with 1 Axes>

### MCMC原理实现过程及其代码

在前面的代码示例中，存在两个问题：
- **我们遍历了所有先验参数的可能性去计算后验分布，然而当参数不止一个时，遍历所有参数非常消耗计算量。**
  
  对应的代码为 `prior_θs = np.linspace(0, 1, 100)`。可以想象，如果参数不止一个，那么prior_θs将不再是一个一维向量。

- **对于复杂的后验分布，公式的分母部分难以计算，所以我们无法得到标准化的后验分布**

  对应代码部分 `np.sum(unstd_posterior)`。可以想象，如果参数不止一个，我们需要对每一个参数都进行求和操作。


使用MCMC的**意义**就在于：

通过采样的方法避免遍历所有的参数值，并且避免计算分母。

因此，MCMC的关键在于**如何在后验分布中进行采样**？

**为了了解采样的概念，我们首先从参数范围中随机进行采样**

我们知道参数(海面覆盖率)的范围为0到1，因此我们可以从0到1中随机抽取参数

In [32]:
samples = st.uniform(0,1).rvs(100) # 我们从一个0到1的均匀分布中采样100次，用samples代表采集到的样本

In [33]:
# 我们也可以通过for循环达到同样的效果
n_iters = 100
samples = {"θ":np.zeros(n_iters)}
for n_iter in range(n_iters):
  samples["θ"][n_iter] = st.uniform(0,1).rvs(1)

In [34]:
plt.hist(samples["θ"])

(array([13.,  5.,  8.,  7.,  9., 13., 13., 10., 11., 11.]),
 array([0.00284246, 0.10198233, 0.20112219, 0.30026206, 0.39940193,
        0.4985418 , 0.59768166, 0.69682153, 0.7959614 , 0.89510127,
        0.99424113]),
 <BarContainer object of 10 artists>)

<Figure size 432x288 with 1 Axes>

我们可以看到，随机采集的参数样本与真实的后验分布差距很大。

### 拒绝采样(reject sampling)

**通过前面学到的*拒绝采样*算法，我们可以优化采集到的样本，使得它更接近后验样本**

具体思路为，我们会利用未标准化的后验，当该后验对应的y轴的值越大时，我们越有可能保留对应的参数样本。

这里我们重新定义计算后验的函数。

其中，我们定义先验分布为正态分布。
并且，避免参数值超过0-1的范围。

In [35]:
def calculate_posterior(θ, data):
    if 0 <= θ <= 1:                           # 避免参数值超过0-1的范围
      prior = st.norm(0.5, 0.1).pdf(θ)        # 定义先验分布是均值为0.5，标准差是0.1的正态分布。
      likelihood  = st.bernoulli(θ).pmf(data).prod()
      unstd_posterior = likelihood * prior
    else:
        unstd_posterior = -np.inf
        
    return unstd_posterior

- 首先，我们从0到1中随机生成一个参数值：`proposal_θ = st.uniform(0,1).rvs(1)`
- 然后，我们计算该参数值的非标准后验：
  
  `unstd_posterior = calculate_posterior(proposal_θ, prior_θ, data)`。
  
  这里面我们的数据为10次抛地球仪，获得7次海面向上；先验服从均值为0.5标准差为0.1的正态分布。
- 最后，我们生成一个随机数，如果非标准后验的值大于该随机数，我们就接受该参数采样，否则我们保持之前的参数采样：
  
  ```
  if unstd_posterior*1000 > st.uniform(0, 1).rvs(1):
        θ = proposal_θ
  else:
        θ = samples['θ'][n_iter-1]
  ```
        
  可以想象，非标准后验的值越大，对应的参数样本越有可能被保留

我们通过一个**练习**来了解，哪些后验参数容易被接受：

这里我们设置拒绝阈限为0.5 `reject_threshold = 0.5`，

当后验参数对应的y轴数值大于该值，我们接受该后验参数 

```
if unstd_posterior > reject_threshold: 
  print("接受的参数：",θ)
```

In [102]:
import numpy as np               # numpy 是专门用于数组运算的包
import scipy.stats as st         # 从scipy.stats里载入分布函数
import matplotlib.pyplot as plt  # matplotlib.pyplot 是专门用于画图的包

########################################################
# 练习
# 请尝试通过 st.uniform(0, 1).rvs(1) 随机生成一个0-1的参数
########################################################
θ = ...

# 根据数据与先验参数计算后验
data = [1,1,1,1,1,0,0,1,1,0]
unstd_posterior = calculate_posterior(θ, data)
unstd_posterior = unstd_posterior*1000 # 因为unstd_posterior太小，我们将其扩大1000倍

# 设定拒绝阈限
reject_threshold = 0.5

print("参数y轴值：",unstd_posterior,"拒绝阈限",reject_threshold)
if unstd_posterior > reject_threshold:
    print("接受的参数：",θ)
else:
    print("拒绝的参数：",θ)

我们通过for循环尝试生成1000个随机的参数(proposal_θ):
```
n_iters = 1000
for n_iter in range(n_iters):
  proposal_θ = st.uniform(0,1).rvs(1)
```

如果参数对应y轴数值的1000倍大于0.5我们接受该参数，否则我们拒绝该参数：
```
reject_threshold = 0.5
if unstd_posterior*1000 > reject_threshold:
  θ = proposal_θ
  samples['θ'][n_iter] = θ
```

In [36]:
data = [1,1,1,1,1,1,1,0,0,0] # 数据与之前相同，10次抛地球仪，我们获得7次海面向上
n_iters = 1000
samples = {"θ":np.zeros(n_iters)}

for n_iter in range(n_iters):
    proposal_θ = st.uniform(0,1).rvs(1)
    unstd_posterior = calculate_posterior(proposal_θ, data)
    
    reject_threshold = 0.5
    if unstd_posterior*1000 > reject_threshold:
        θ = proposal_θ
    else:
        θ = samples['θ'][n_iter-1]

    samples['θ'][n_iter] = θ

我们可以对该分布进行统计推断，如同上一次课程我们对后验分布进行统计推断一样。

In [37]:
az.plot_posterior(samples) # 绘制样本的分布

<AxesSubplot:title={'center':'θ'}>

<Figure size 432x288 with 1 Axes>

In [38]:
az.summary(samples) # 计算样本的均值与HDI

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
θ  0.556  0.106   0.401    0.737      0.006    0.004     264.0     238.0   

   r_hat  
θ    NaN

从结果看，采集的参数大多位于0.5-0.7, 这符合真实后验分布的结果。

重要性采样有两个问题：
1. 我们采样参数的过程是随机的`proposal_θ = st.uniform(0,1).rvs(1)`，这样采样参数在分布中peak位置的数量较少，导致采样的效率低。
2. 如果我们设置的**拒绝阈限**太大，那么我们采集到的数据大多都会被拒绝。

通过**练习**了解不同**拒绝阈限**对接受的样本数量的影响

In [ ]:
n_iters = 1000
samples_accept = []
samples_reject = []

########################################################
# 练习
# 请尝试设置 reject_threshold 为0-1的任何值
########################################################
reject_threshold = ...

for n_iter in range(n_iters):
    proposal_θ = st.uniform(0,1).rvs(1)
    unstd_posterior = calculate_posterior(proposal_θ, data)
    
    if unstd_posterior*1000 > reject_threshold:
        samples_accept += [proposal_θ[0]]
    else:
        samples_reject += [proposal_θ[0]]

accept_number = [len(samples_accept), len(samples_reject)]
print("接受的样本数量：", accept_number[0], "拒绝的样本数量：", accept_number[1])

fig,ax = plt.subplots()
ax.bar(["accept","reject"],accept_number)
plt.show()

**MCMC Metropolis 算法**

为了提高采样的效率，减少被拒绝的采样，我们可以使用 MCMC中的 Metropolis 算法。

其与接受拒绝采样的差别在于，我们接受采样的概率与之前的采样的后验有关。

`ratio = unstd_posterior1 / unstd_posterior0`

并且，我们每次采样不是随机采样，而是在当前后验参数附近采样：

`proposal_θ1 = st.norm(proposal_θ0,0.5).rvs(1)`

这种根据先前采样的建议(proposal)得到的采样，称为proposal参数。

In [42]:
n_iters = 1000
samples = {"θ":np.zeros(n_iters)}
proposal_θ1 = 0          # 为当前迭代下的proposal_θ
proposal_θ0 = 0          # 为上一个迭代的proposal_θ
unstd_posterior1 = 0.001 # 为当前迭代下的unstd_posterior
unstd_posterior0 = 0.001 # 为上一个迭代的unstd_posterior

for n_iter in range(n_iters):
    proposal_θ1 = st.norm(proposal_θ0,0.5).rvs(1)
    unstd_posterior1 = calculate_posterior(proposal_θ1, data)
    
    ratio = unstd_posterior1 / unstd_posterior0

    if ratio > st.uniform(0, 1).rvs(1):
        proposal_θ0 = proposal_θ1
        unstd_posterior0 = unstd_posterior1
        
    samples['θ'][n_iter] = proposal_θ0

In [43]:
az.plot_posterior(samples) # 绘制样本的分布

<AxesSubplot:title={'center':'θ'}>

<Figure size 432x288 with 1 Axes>

In [44]:
az.summary(samples) # 计算样本的均值与HDI

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
θ  0.539  0.099   0.383    0.682      0.008    0.006     166.0     169.0   

   r_hat  
θ    NaN

可以看到，我们顺利得到了参数的后验分布，并且该结果比重要性采样算法的结果更好。

有了该采样样本，我们就可以用上一节课学习到的统计推断方法分析该采样分布了

同样，我们可以比较 proposal参数在metroplis算法中的接受频率。

In [55]:
n_iters = 1000
samples_accept = []
samples_reject = []

for n_iter in range(n_iters):
    proposal_θ1 = st.norm(proposal_θ0,0.5).rvs(1)
    unstd_posterior1 = calculate_posterior(proposal_θ1, data)
    
    ratio = unstd_posterior1 / unstd_posterior0

    if ratio < st.uniform(0, 1).rvs(1):
        samples_accept += [proposal_θ[0]]
    else:
        samples_reject += [proposal_θ[0]]

accept_number = [len(samples_accept), len(samples_reject)]
print("接受的样本数量：", accept_number[0], "拒绝的样本数量：", accept_number[1])

fig,ax = plt.subplots()
ax.bar(["accept","reject"],accept_number)
plt.show()

接受的样本数量： 824 拒绝的样本数量： 176


<Figure size 432x288 with 1 Axes>

可以看到，由于metroplis算法的接受效率更好。

需要注意的是，通过MCMC采样得到的是从后验分布中采样得到的参数样本，而不是关于后验分布的函数。

后验分布参数样本相比于后验分布函数的优点在于：
- 方便计算后验分布的平均值等统计值

缺点在于：
- 如果样本无法代表后验分布，基于后验分布的统计推断也会出错

### pymc

显然通过写for循环来实现MCMC算法还是过于复杂，我们下节课将结合 pymc工具包，简化以上计算。

In [ ]:
import pymc3 as pm

data = [1,1,1,1,1,0,0,1,1,0]
with pm.Model() as model:
    # 设定先验
    θ = pm.Normal("θ", 0.5, 0.1)
    # 设定似然函数与数据
    pm.Binomial("samples", n=1, p=θ, observed=data)
    # Sample from the posterior distribution
    samples = pm.sample(1000, return_inferencedata=True)

In [52]:
az.plot_dist(samples.posterior["θ"])

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [θ]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 4 seconds.


<AxesSubplot:>

<Figure size 432x288 with 1 Axes>

In [53]:
az.summary(samples)

mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
θ  0.557  0.081   0.401    0.706      0.002    0.001    1665.0    2819.0   

   r_hat  
θ    1.0

从pymc的采样结果来看，同样采集1000个样本，pymc的后验分布明显更加平滑和准确。

这提示我们，即使我们知道采样器的算法和原理，但我们自己写的代码仍然和专业工具包存在差异。

因此更重要的是，我们的学习目标是如何利用优秀的算法和工具完成数据分析，而不是自己研究如何创造算法和工具

总结，MCMC算法的专业术语，包括：

- MCMC samplers：由于MCMC算法的本质是对后验分布进行采样，因此该方法也被叫做MCMC采样器(samplers)。
- iteration，draws，samples：MCMC采样器在采样过程中需要进行循环，每个循环称为一个iteration，每个iteration中采集的参数称作样本samples，也可以叫做draws，就像从抽屉里抽出来一样。
- proposal dsitribution：即参数采样的过程不再是随机采样，而是根据建议分布(proposal dsitribution)进行采样。
- chain：在使用MCMC算法时，我们会同时运行多个“循环”，每一个循环为一条马尔科夫链(chain)。运行多条链的目的在于，可以更高效的采集更多的样本，也可以避免单条链采到的样本不能代表后验分布的影响。
